In [1]:
import findspark
findspark.init('/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('mytreeU').getOrCreate()

21/11/30 08:23:37 WARN Utils: Your hostname, venkat-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/11/30 08:23:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/30 08:23:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/30 08:23:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
data = spark.read.csv('College.csv', inferSchema = True, header = True)

In [3]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [8]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.feature import StringIndexer

In [11]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')
output_fixed = indexer.fit(output).transform(output)

In [12]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [13]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [14]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [16]:
from pyspark.ml import Pipeline

In [17]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [18]:
dtc_model = dtc.fit(train_data)
rfc_model=rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [19]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [22]:
print('Decision Tree')
print(my_binary_eval.evaluate(dtc_preds))

Decision Tree
0.9171914829809567


In [23]:
print('Random Forest')
print(my_binary_eval.evaluate(rfc_preds))

Random Forest
0.9713975108711949


In [26]:
print('Gradient Boosted')
print(my_binary_eval.evaluate(gbt_preds))

Gradient Boosted
0.9537411905832958


In [30]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')
print('Gradient Boosted2')
print(my_binary_eval2.evaluate(gbt_preds))

Gradient Boosted2
0.9072199730094467


In [29]:
gbt_preds.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|PrivateIndex|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|[150.0,130.0,88.0...|         0.0|[1.54542886317630...|[0.95651405607397...|       0.0|
|[152.0,128.0,75.0...|         0.0|[1.54542886317630...|[0.95651405607397...|       0.0|
|[167.0,130.0,46.0...|         0.0|[1.47853699358929...|[0.95059676190037...|       0.0|
|[213.0,155.0,75.0...|         0.0|[0.94151405741989...|[0.86795855495344...|       0.0|
|[222.0,185.0,91.0...|         0.0|[1.54542886317630...|[0.95651405607397...|       0.0|
|[235.0,217.0,121....|         0.0|[1.54542886317630...|[0.95651405607397...|       0.0|
|[247.0,189.0,100....|         0.0|[1.54542886317630...|[0.95651405607397...|       0.0|
|[257.0,183.0,109....|         0.0|[1.54542886317630...|[0.95651405607397...|       0.0|
|[261.0,192.0,111....

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [33]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [35]:
rfc_acc = acc_eval.evaluate(rfc_preds)
print(rfc_acc)

0.9156626506024096
